In [19]:
import helpers
import numpy as np
from scipy.cluster.hierarchy import fcluster, linkage
from sklearn.metrics import adjusted_rand_score,silhouette_score
from k_medoid_clustering import KMedoid
from bernoulli_mixture_model import BMM
import timeit
from Bio import AlignIO
from Bio.Phylo import to_networkx, draw_ascii
from Bio.Phylo.TreeConstruction import ParsimonyTreeConstructor, ParsimonyScorer, NNITreeSearcher
from Bio.Phylo.TreeConstruction import DistanceCalculator, DistanceTreeConstructor, ParsimonyTreeConstructor, ParsimonyScorer, NNITreeSearcher
from Bio import Phylo

In [20]:
raw_scs_data = "/home/laurynas/workspace/individual_project/data/hou/snv.csv"

In [21]:
helper = helpers.Helper()

In [22]:
number_of_clusters = 6

In [23]:
number_of_iterations = 15

In [24]:
unique_rows, full_or_data, full_info = helper.process_single_cell_data_file(raw_scs_data)
# print full_or_data
bmm = BMM(k_clusters=number_of_clusters
             , unique_rows=unique_rows, full_data_dict=full_or_data, full_info=full_info,
          number_of_iterations=number_of_iterations)
data, predicted_labels = bmm.do_clustering()
predicted_labels = np.asarray(predicted_labels)

ready_gen, labels_in_clusters = helper.get_single_cell_genotypes(data, full_or_data)

write_dir = "./single_cell/cluster_genotypes.phy"
target = open(write_dir, 'w+')

target.write(str(len(ready_gen)) + " " + str(len(ready_gen[0])) + "\n")

for index, el in enumerate(ready_gen):
    target.write(("Cluster "+str(index)).ljust(10) + el.replace("NA","N") + "\n")
target.close()

aln = AlignIO.read(write_dir, 'phylip')
dtc = DistanceTreeConstructor(DistanceCalculator("identity"),
                              "nj")
nj_tree = dtc.build_tree(aln)
scorer = ParsimonyScorer()
searcher = NNITreeSearcher(scorer)
constructor_parsimony = ParsimonyTreeConstructor(searcher, starting_tree = nj_tree)
pars_tree = constructor_parsimony.build_tree(aln)
draw_ascii(pars_tree)

Phylo.draw_graphviz(pars_tree, prog="dot")

plt.show()

getting genotypes:  0 50
getting genotypes:  1 2
getting genotypes:  2 2
getting genotypes:  3 2
getting genotypes:  4 2
getting genotypes:  5 2
  __________________________________________________________________ Cluster 4
 |
 |                   __________________________________ Cluster 5
_|          ________|
 |         |        |  ___________________ Cluster 0
 |         |        |_|
 |_________|          |    ______________________________ Cluster 1
           |          |___|
           |              |_________________________________ Cluster 2
           |
           |_________________________________ Cluster 3



MissingPythonDependencyError: Install PyGraphviz or pydot if you want to use draw_graphviz.